In [1]:
%cd ..
%pwd

/home/naveen/mlops/wine-quality-mlops


'/home/naveen/mlops/wine-quality-mlops'

In [2]:
import joblib
from pathlib import Path

import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

In [3]:
train_data_file_path: Path = Path('data/transformed/train.csv')
test_data_file_path: Path = Path('data/transformed/test.csv')
target_column: str = 'quality'
model_path = Path('models/ElasticNet')
train_data = pd.read_csv(train_data_file_path)
test_data = pd.read_csv(test_data_file_path)

train_X = train_data.drop([target_column], axis=1)
test_X = test_data.drop([target_column], axis=1)
train_y = train_data[target_column]
test_y = test_data[target_column]

SyntaxError: cannot use double starred expression here (3239650485.py, line 1)

In [13]:
# Tune hyperparameters
params = {
    'alpha': 0.5,
    'l1_ratio': 0.3,
}
# Train model with new params
from wine import model
model.train(train_data_file_path=train_data_file_path,
            target_column=target_column, **params,
            model_path=model_path)

# Predict on test set
loaded_model = joblib.load(model_path)
y_pred = loaded_model.predict(test_X)

# Calculate metrics
rmse = root_mean_squared_error(y_true=test_y, y_pred=y_pred)
mae = mean_absolute_error(y_true=test_y, y_pred=y_pred)
r2 = r2_score(y_true=test_y, y_pred=y_pred)

metrics = {
    'rmse': rmse,
    'mae': mae,
    'r2': r2,
}

In [7]:
rmse

0.8000726678127325

In [8]:
import os

os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/agrawalnaveen/wine-quality-mlops.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'agrawalnaveen'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'afcc5e09fde72bc9573fea2cdd1624185b74be16'

In [14]:
# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="https://dagshub.com/agrawalnaveen/wine-quality-mlops.mlflow")
# Create a new MLflow Experiment
mlflow.set_experiment("Elastic Net Model")
# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metrics(metrics)

In [35]:
import json
metrics_file_path = Path('models/metrics.json')
with open(metrics_file_path, 'w') as metrics_file:
    json_object = json.dump(metrics, metrics_file)

In [27]:
print(json_object)

{"rmse": 0.7701641420705332, "mae": 0.6073926761485033, "r2": 0.1861445336806553}


In [39]:
with open(metrics_file_path, 'r') as metrics_file:
        metrics = json.load(metrics_file)

In [ ]:
export MLFLOW_TRACKING_URI=https://dagshub.com/agrawalnaveen/wine-quality-mlops.mlflow \
export MLFLOW_TRACKING_USERNAME=agrawalnaveen \
export MLFLOW_TRACKING_PASSWORD=afcc5e09fde72bc9573fea2cdd1624185b74be16 \
python script.py